In [ ]:
!pip install openai pdfplumber openpyxl
from openai import OpenAI
import pdfplumber
import openpyxl
from openpyxl import Workbook


!pip install openai beautifulsoup4 openpyxl

### Python Script

import openai
from bs4 import BeautifulSoup
import openpyxl
from openpyxl import Workbook
import json


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [3]:
# Set your OpenAI API key
client = OpenAI(api_key="XXX")

# Function to extract text from PDF
def extract_text_from_pdf(pdf_file_path):
    with pdfplumber.open(pdf_file_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"
    return full_text

# Function to use GPT-4 for parsing financial data
def parse_financial_data(text, model='gpt-4'):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that analyzes and extracts financial data from text. Return the data as a JSON object with the keys 'Revenue', 'Gross Profit', 'Operating Expenses', and 'Net Income', each containing a dictionary of dates and corresponding values. Ensure all values are enclosed in double quotes."}, # Modified prompt to ensure correct JSON format
        {"role": "user", "content": text}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=2000,
        temperature=0.3
    )

    return response.choices[0].message.content.strip()

# Function to create an Excel file with financial data
def create_excel(financial_data, output_file):
    wb = Workbook()
    ws = wb.active
    ws.title = "Financial Data"

    # Define headers
    headers = ["Revenue", "Gross Profit", "Operating Expenses", "Net Income"]
    for index, header in enumerate(headers, start=1):
        ws.cell(row=index+1, column=1, value=header)  # Placing headers in the first column

    # Fill in financial data
    if isinstance(financial_data, dict): # Check if financial_data is a dictionary
        dates = list(set().union(*[dates for dates in financial_data.values()])) # Get all unique dates
        for col_index, date in enumerate(dates, start=2):
            ws.cell(row=1, column=col_index, value=date)  # Placing dates in the first row
            for row_index, header in enumerate(headers, start=1):
                # Access the financial data for the specific header and date
                value = financial_data.get(header, {}).get(date)
                if value is not None:
                    ws.cell(row=row_index+1, column=col_index, value=value)
    else:
        print("Error: Financial data is not in the expected dictionary format.")

    wb.save(output_file)

if __name__ == "__main__":
    pdf_file_path = '/content/cdns2.pdf'  # Replace with your PDF file path
    output_excel_file = 'financial_data.xlsx'

    # Step 1: Extract text from PDF
    pdf_text = extract_text_from_pdf(pdf_file_path)

    # Step 2: Parse the text using GPT-4 to get structured financial data
    financial_data_json = parse_financial_data(pdf_text, model='gpt-4')

    # Convert JSON string to Python dictionary
    import json
    try:
        financial_data = json.loads(financial_data_json)
    except json.JSONDecodeError:
        print("Error: Could not decode JSON response from GPT-4. Please check the response format.")
        financial_data = None # Set financial_data to None if JSON decoding fails

    # Step 3: Create an Excel file with the financial data
    if financial_data: # Only proceed if financial_data is valid
        create_excel(financial_data, output_excel_file)
        print(f"Financial data has been written to {output_excel_file}")

Financial data has been written to financial_data.xlsx


In [4]:
# Set your OpenAI API key
client = OpenAI(api_key="XXX")

# Function to extract text from PDF
def extract_text_from_pdf(pdf_file_path):
    with pdfplumber.open(pdf_file_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"
    return full_text

# Function to use GPT-4 for parsing financial data
def parse_financial_data(text, model='gpt-4'):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that analyzes and extracts financial data from text. Return the data as a JSON object with the keys 'Revenue', 'Gross Profit', 'Operating Expenses', and 'Net Income'."},
        {"role": "user", "content": text}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=2000,
        temperature=0.3
    )

    return response.choices[0].message.content.strip()

def create_excel(financial_data, output_file):
    wb = Workbook()
    ws = wb.active
    ws.title = "Financial Data"

    # Define headers
    headers = ["Revenue", "Gross Profit", "Operating Expenses", "Net Income"]
    for index, header in enumerate(headers, start=1):
        ws.cell(row=1, column=index, value=header)

    # Fill in financial data
    for index, header in enumerate(headers, start=1):
        if header in financial_data:
            ws.cell(row=2, column=index, value=financial_data.get(header))

    wb.save(output_file)

if __name__ == "__main__":
    pdf_file_path = '/content/cdns2.pdf'  # Replace with your PDF file path
    output_excel_file = 'financial_data.xlsx'

    # Step 1: Extract text from PDF
    pdf_text = extract_text_from_pdf(pdf_file_path)

    # Step 2: Parse the text using GPT-4 to get structured financial data
    financial_data_json = parse_financial_data(pdf_text, model='gpt-4')

    # Convert JSON string to Python dictionary
    import json
    financial_data = json.loads(financial_data_json)


    # Step 3: Create an Excel file with the financial data
    create_excel(financial_data, output_excel_file)

    print(f"Financial data has been written to {output_excel_file}")

Financial data has been written to financial_data.xlsx
